In [10]:
library("proxy")
library("dtw")
library("rucrdtw")

# Test of fastDTW

Our main function here is similarity_TS. 
* Input:
    * Data Matrix of size $(\sum_{i}^{n}T_{i}) \times p$
    * One of the following
        * len_time: A sequence of $T_{i}$, i=1,2...n; $T_{i}$ are integers here.
        * time_cut : The first time of each unit $(1,T_{1}+1,T_{1}+T_{2}+1,...)$ (not implemented yet)
    * type: The method used when computing distance. This includes "L2" and so on (others not implemented)
        
* Method:
    * Z score normalize each time series, say time 1 to $T_{1}$ for feature 1.
    * Calculate the distance measure according to type with each of n time series, denoted by $d_{i}$
    * Calculate the average estimate of distance of time series as 
    $$d = \frac{\sum_{i=1}^{n}d_{i}T_{i}}{\sum_{i=1}^{n}T_{i}}$$
    * The similarity between the two time series is given by $$\frac{1}{1+d}$$
    
* Output: The $\textbf{similarity}$ (inverse relationship with distance) matrix between features (time series generator) with entries in $[0,1]$. The reason for making it smaller than 1 is that we will raise it to the power $\beta$ later.
* Remark: 
    * (Important!) If the original data has certain time series with the same values, normalized_X will have NA values because in the Z score normalization, $\sigma = 0$. This case should be solved in the data processing step.
    * Maybe later we need the exact time (instead of the index of a certain time in one time series) since some measure requires exact time.
    * After this step, the similarity is raised to the power $\beta$ and then used for calculate TOM

# Functions

In [33]:
# Normalize each time series (not a column)
normalize_TS = function(X,len_time){
    normalized_X = matrix(0,dim(X)[1],dim(X)[2])
    flag = 1 # record the first index of the current time series
    for (t in len_time){
        normalized_X[flag:(flag+t-1),] = scale(X[flag:(flag+t-1),])
        flag = flag + t
    }
    return (normalized_X)
}

# The main function                                                 
# compute similarity using different measure
similarity_TS = function(X,len_time,type){
    p = dim(X)[2] # num of features
    n = length(len_time) # number of samples
    
    ## type = cor, no need to normalize ##
    if (type == "cor"){
        # first compute distance matrix
        if (n==1){
            cor_mat = cor(X)
        }else{
            # cor_mat can be computed like a vector! but I have to create it
            cor_mat = cor(X[1:len_time[1],])*len_time[1]
            flag = 1+len_time[1] # the first time index of the current sample
            for (sample in 2:n){
                cor_mat = (cor_mat + 
            cor(X[flag:(flag+len_time[sample]-1),])*len_time[sample])
                
                flag = flag+len_time[sample]
            }
            cor_mat = cor_mat/(sum(len_time))
        }     
           
        return (cor_mat)
        } # end "cor"
    
     ## type = fastDTW, no need to normalize ##
    if (type == "fastDTW"){
        # pre-allocate memory
        dist_mat = matrix(0,p,p)
        if (n==1){
            for (i in 1:(p-1)){
                for (j in (i+1):p){
                    dist_mat[i,j] = ucrdtw_vv(X[,i], X[,j], 0.05)$distance
                }
            }
        }else{
            flag = 1 # the first time index of the current sample
            for (sample in 1:n){
                for (i in 1:(p-1)){
                    for (j in (i+1):p){
                        dist_mat[i,j] = (dist_mat[i,j]
                        +(ucrdtw_vv(X[flag:(flag+len_time[sample]-1),i], 
                                                  X[flag:(flag+len_time[sample]-1),j],
                                                  0.05)$distance)*len_time[sample])
                    }
                }                
                flag = flag+len_time[sample]
            }
            dist_mat = dist_mat/(sum(len_time))
        }
        # complete the matrix (diag=0 automatically)
        dist_mat[lower.tri(dist_mat)] = t(dist_mat)[lower.tri(dist_mat)]
        
        # transform dist to similarity
        sim_matrix = 1/(1+dist_mat)
        
        return (sim_matrix)
        } # end "fastDTW"
    
    #### data Z score normaliztion #####
    X = normalize_TS(X,len_time) # normalize time series
    #check for missing value (may due to sigma=0)
    tmp = sum(is.na(X))
    if(tmp>0){
        stop(sprintf("There are %d missing values in normalized X",tmp))
    }
    ### end normalization ###
    
    
    # "L2" using dist(M) which computes dist between rows of M
    if (type == "L2"|type == "Euclidean"){
        # first compute distance matrix
        if (n==1){
            dist_obj = dist(t(X),method = "euclidean")
        }else{
            # dist_obj can be computed like a vector! but I have to create it
            dist_obj = dist(t(X[1:len_time[1],]))*len_time[1]
            flag = 1+len_time[1] # the first time index of the current sample
            for (sample in 2:n){
                dist_obj = (dist_obj + 
            dist(t(X[flag:(flag+len_time[sample]-1),]))*len_time[sample])
                
                flag = flag+len_time[sample]
            }
            dist_obj = dist_obj/(sum(len_time))
        }     
    
        # transform dist to similarity
        sim_obj = 1/(1+dist_obj)
        # similarity matrix
        sim_matrix = as.matrix(sim_obj)
        diag(sim_matrix) = 1
        
        return (sim_matrix)
        } # end "L2"
    
    # "dtw" using dtwDist(M) which computes dist between rows of M
    if (type == "dtw" | type == "DTW"){
        # first compute distance matrix
        if (n==1){
            dist_obj = dtwDist(t(X))
        }else{
            # dist_obj can be computed like a vector! but I have to create it
            dist_obj = dtwDist(t(X[1:len_time[1],]))*len_time[1]
            flag = 1+len_time[1] # the first time index of the current sample
            for (sample in 2:n){
                dist_obj = (dist_obj + 
            dtwDist(t(X[flag:(flag+len_time[sample]-1),]))*len_time[sample])
                
                flag = flag+len_time[sample]
            }
            dist_obj = dist_obj/(sum(len_time))
        }     
    
        # transform dist to similarity
        sim_obj = 1/(1+dist_obj)
        # similarity matrix
        sim_matrix = as.matrix(sim_obj)
        diag(sim_matrix) = 1
        
        return (sim_matrix)
        } # end "dtw"
    
    
    stop("The type is not availble")
    
    }

# For Testing

In [34]:
# For test1 (small)
# data matrix n=2 p=3
X = matrix(0,5,3)
len_time = c(3,2) # T1=3, T2=2
X[1:3,1] = c(1,1,1.1)
X[1:3,2] = c(2,2,2.2)
X[1:3,3] = c(1,0.5,1.5)
X[4:5,1] = c(1.2,1.3)
X[4:5,2] = c(2,2.2)
X[4:5,3] = c(5,15)
X

1.0,2.0,1.0
1.0,2.0,0.5
1.1,2.2,1.5
1.2,2.0,5.0
1.3,2.2,15.0


In [25]:
X = matrix(runif(15),5,3)

In [35]:
similarity_TS(X,len_time,"DTW")

1.0000000,1.0000000,0.4903811
1.0000000,1.0000000,0.4903811
0.4903811,0.4903811,1.0000000


In [38]:
# For test3 (bigger)
n = 100 # samples
T = 5 # time each samples
p = 100 # features
len_time = rep(T,n)
X = matrix(rnorm(n*T*p),n*T,p) # (n*T)*p

In [39]:
start_time <- Sys.time()
sim_mat = similarity_TS(X,len_time,"fastDTW")
sim_mat
end_time <- Sys.time()

end_time - start_time

1.0000000,0.2527188,0.2513648,0.2499762,0.2437693,0.2514962,0.2541645,0.2470415,0.2426133,0.2438680,...,0.2494234,0.2547877,0.2477149,0.2396427,0.2410835,0.2477913,0.2471473,0.2446969,0.2489592,0.2445894
0.2527188,1.0000000,0.2478862,0.2379116,0.2530776,0.2415487,0.2411045,0.2486544,0.2433319,0.2498186,...,0.2401578,0.2458292,0.2544881,0.2378517,0.2503015,0.2511628,0.2547366,0.2397788,0.2519206,0.2410519
0.2513648,0.2478862,1.0000000,0.2496661,0.2528613,0.2484438,0.2537876,0.2438618,0.2435716,0.2549035,...,0.2458890,0.2534106,0.2447459,0.2525233,0.2497567,0.2492204,0.2420892,0.2465941,0.2518880,0.2513914
0.2499762,0.2379116,0.2496661,1.0000000,0.2482838,0.2497044,0.2576820,0.2489941,0.2491325,0.2484126,...,0.2428195,0.2527735,0.2557563,0.2441309,0.2372175,0.2477878,0.2470667,0.2560155,0.2419017,0.2462462
0.2437693,0.2530776,0.2528613,0.2482838,1.0000000,0.2532800,0.2472725,0.2508542,0.2503526,0.2394097,...,0.2498486,0.2374170,0.2496964,0.2484246,0.2537513,0.2548924,0.2491763,0.2425134,0.2434957,0.2408156
0.2514962,0.2415487,0.2484438,0.2497044,0.2532800,1.0000000,0.2489663,0.2543534,0.2542351,0.2461543,...,0.2533579,0.2491404,0.2459662,0.2500293,0.2473434,0.2443081,0.2485986,0.2358924,0.2601262,0.2542304
0.2541645,0.2411045,0.2537876,0.2576820,0.2472725,0.2489663,1.0000000,0.2599371,0.2508128,0.2573523,...,0.2495628,0.2580801,0.2481473,0.2589349,0.2494780,0.2518604,0.2468248,0.2396656,0.2393965,0.2560202
0.2470415,0.2486544,0.2438618,0.2489941,0.2508542,0.2543534,0.2599371,1.0000000,0.2483339,0.2417081,...,0.2474243,0.2465150,0.2614279,0.2496782,0.2439158,0.2470669,0.2543369,0.2455682,0.2415107,0.2506251
0.2426133,0.2433319,0.2435716,0.2491325,0.2503526,0.2542351,0.2508128,0.2483339,1.0000000,0.2419118,...,0.2494840,0.2533481,0.2566368,0.2649960,0.2489995,0.2508339,0.2453473,0.2592739,0.2485431,0.2418545
0.2438680,0.2498186,0.2549035,0.2484126,0.2394097,0.2461543,0.2573523,0.2417081,0.2419118,1.0000000,...,0.2487725,0.2523671,0.2420967,0.2503292,0.2424425,0.2533885,0.2430426,0.2472624,0.2415442,0.2552432
0.2460570,0.2458525,0.2514143,0.2437311,0.2415306,0.2451330,0.2520274,0.2568384,0.2510940,0.2478116,...,0.2470759,0.2557629,0.2490931,0.2551465,0.2403941,0.2468950,0.2543475,0.2414408,0.2415801,0.2598602


Time difference of 56.95113 secs

In [54]:
similarity_TS(X,len_time,"L2")

1,2,3
1.0000000,1.0000000,0.6948157
1.0000000,1.0000000,0.6948157
0.6948157,0.6948157,1.0000000


# Test of speed

In [48]:
system.time(dist(t(X),method="tsDistances",distance="euclidean"))

ERROR: Error in .get_entry_index(name, stop_if_missing): Entry "tsDistances" not in registry.


Timing stopped at: 0 0 0


In [24]:
system.time(dist(t(X)))

   user  system elapsed 
   0.03    0.00    0.03 

In [23]:
system.time(TSDatabaseDistances(t(X),distance="euclidean"))

   user  system elapsed 
   4.79    0.14    5.89 